#### PREPARE A BASE MODEL

In machine learning, transfer learning is the process of using a model that has already been trained on a data set to carry out a different but related task. <br> 
The model that has been trained on the data set is called the base model. In some cases, transfer learning involves the retraining of the base model, and in other cases, it involves the creation of a new model atop the base model.

In [1]:
import os

%pwd

'd:\\work\\cifar100classifier-deep-learning\\research'

In [2]:
os.chdir('../')

%pwd

'd:\\work\\cifar100classifier-deep-learning'

In [3]:
import sys

sys.path.append('src')

%pwd

'd:\\work\\cifar100classifier-deep-learning'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [5]:
from CNNClassifier.constants import *
from CNNClassifier.utils.utilities import read_yaml, create_directory

In [6]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directory([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        create_directory([config.root_dir])
        
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir = Path(config.root_dir),
            base_model_path = Path(config.base_model_path),
            updated_base_model_path = Path(config.updated_base_model_path),
            params_image_size = self.params.IMAGE_SIZE,
            params_learning_rate = self.params.LEARNING_RATE,
            params_include_top = self.params.INCLUDE_TOP,
            params_weights = self.params.WEIGHTS,
            params_classes = self.params.CLASSES 
        )

        return prepare_base_model_config 

In [8]:
# import os
# import urllib.request as request
# from zipfile import ZipFile
import tensorflow as tf

In [9]:
# config.yaml -> config_entity.py -> configuration.py -> components -> pipeline

class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
    
    def conv_base_model(self):
        # Compiles a model integrated with VGG16 pretrained layers
        # input_shape: tuple - the shape of input images (width, height, channels)
        # n_classes: int - number of classes for the output layer
        # optimizer: string - instantiated optimizer to use for training. Defaults to 'RMSProp'
        # fine_tune: int - The number of pre-trained layers to unfreeze. If set to 0, all pretrained layers will freeze during training
        # Include_top is set to False, in order to exclude the model's fully-connected layers.
        # Pretrained convolutional layers are loaded using the Imagenet weights.

        self.model = tf.keras.applications.vgg16.VGG16(
            include_top = self.config.params_include_top,
            weights = self.config.params_weights,
            input_shape = self.config.params_image_size
        )

        self.save_model(path=self.config.base_model_path, model=self.model)

    @staticmethod
    def _prepare_full_model(model, n_classes, optimizer='RMSProp', freeze_till=0):
        # Defines how many layers to freeze during training.
        # Layers in the convolutional base are switched from trainable to non-trainable
        # depending on the size of the fine-tuning parameter.
        if freeze_till > 0:
            for layer in model.layers[:-freeze_till]:
                layer.trainable = False
        else:
            for layer in model.layers:
                layer.trainable = False
        # Create a new 'top' of the model (i.e. fully-connected layers).
        # This is 'bootstrapping' a new top_model onto the pretrained layers.
        top_model = model.output
        top_model = tf.keras.layers.Flatten(name="flatten")(top_model)
        top_model = tf.keras.layers.Dense(4096, activation='relu')(top_model)
        top_model = tf.keras.layers.Dense(1072, activation='relu')(top_model)
        top_model = tf.keras.layers.Dropout(0.2)(top_model)
        output_layer = tf.keras.layers.Dense(n_classes, activation='softmax')(top_model)
        # Group the convolutional base and new fully-connected layers into a Model object.
        full_model = tf.keras.models.Model(inputs=model.input, outputs=output_layer)
        # Compiles the model for training.
        full_model.compile(
            optimizer=optimizer,
            loss = tf.keras.losses.CategoricalCrossentropy(),
            metrics = ["accuracy"] 
        )
        full_model.summary()
        return full_model
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model = self.model,
            n_classes = self.config.params_classes,
            optimizer = tf.keras.optimizers.Adam(learning_rate=self.config.params_learning_rate),
            freeze_till = 0
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)
        
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [11]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config = prepare_base_model_config)
    prepare_base_model.conv_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2023-10-21 20:59:34,619: INFO: utilities]: yaml file: configs\config.yaml loaded successfully
[2023-10-21 20:59:34,624: INFO: utilities]: yaml file: params.yaml loaded successfully
[2023-10-21 20:59:34,626: INFO: utilities]: create directory at: artifacts
[2023-10-21 20:59:34,628: INFO: utilities]: create directory at: artifacts/prepare_base_model


[2023-10-21 20:59:35,231: WARNING: saving_utils]: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                          